In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold

## Problem Setup:

This example is taken from https://arxiv.org/abs/2107.00681 by Hines, Dukes, Diaz-Ordaz, and Vansteelandt (2021).

$\psi(P_0) = \mathbb{E}[Y|X=x]$  is our target estimand - it is the conditional outcome mean.

i.e. $$\psi(P_0) = \int y \frac{f_{y,x}(y,x)}{f_x(x)} dy $$


If we assume the distribution from which the densities $f$ derive has been perturbed by a point mass at $(\tilde x, \tilde y)$ then:

i.e. $$\psi(P_t) = \int y \frac{f_{y,x,t}(y,x)}{f_{x,t}(x)} dy $$

where $f_{y,x,t}(y,x)$ and $f_{x,t}(x)$ are the joint and marginal densities of $(Y,X)$ and $(X)$ respectively, under the 'parametric submodel' $$P_t = t\delta_x(\tilde x) + (1-t)P_o$$ and where $\delta_x(\tilde x)$ denotes the Dirac delta function s.t. it gives the density of a point mass at $\tilde x$, is zero everywhere else, and integrates to 1.

For our densities we therefore have that:

$$f_{y,x,t}(x,y) =  t \delta_{x,y}(\tilde x, \tilde y) +  (1-t)f_{x,y}(x,y)$$

and

$$f_{x,t}(x) =  t \delta_{x}(\tilde x) +  (1-t)f_{x}(x)$$

For some single observation $\tilde o$, the influence function can be written as:

$$\phi(\tilde o, P) = \left. \frac{d\psi(P_t)}{dt} \right \vert_{t=0} = \int y \left . \frac{d}{dt} \right \vert_{t=0}\frac{f_{x,y,t}(x,y)}{f_{x,t}(x)}dy$$

Following the quotient rule:
$$\phi(\tilde o, P) = \int\left .\frac{y}{f(x)} \frac{df_{x,y,t}(x,y)}{dt} \right \vert_{t=0} dy  - \int\left .\frac{ydf_{x,t}(x)}{dt} \frac{f_{x,y,t}(x,y)}{f(x)^2} \right \vert_{t=0} dy$$

Now $$\left. \frac{df_{y,x,t}(x,y)}{dt}\right \vert_{t=0} =  \delta_{x,y}(\tilde x, \tilde y) -f_{x,y}(x,y)$$

and 
$$\left. \frac{df_{x,t}(x)}{dt} \right \vert_{t=0} =  \delta_{x}(\tilde x) -f_{x}(x)$$

Therefore, with some manipulation and remembering that $\delta_{x,y}(\tilde x, \tilde y) = \delta_{x}(\tilde x) \delta_{y}(\tilde y) $

$$ \phi(\tilde o, P) = \frac{\delta_{x}(\tilde x) }{f(\tilde x)} \left( \tilde y - \mathbb{E}[Y|X=\tilde x]\right )$$

This is fine if $X$ is discrete, since in this case $\delta_{x}(\tilde x)$ is the indicator function $\mathbb{1}_x(\tilde x)$ which equals 1 when $x=\tilde x$.

Finally we want to update our initial estimate as:

$$ \psi(P) \approx \psi(P_n) + \frac{1}{N}\sum_{i=0}^N\phi(\tilde o_i, P_n)$$

which follows the Von Mises process for approximating a functional at an unknown distribution as a sum of terms involving the functional at the current distribution plus some derivatives (in our case, pathwise derivative) and higher order terms. 

In [2]:
# define a function that returns p(X=x) given the PMF p(X) and space of possible X
def pmf(x, cats, p_cats):
    ''' takes in set of possible outcomes 'cats' and corresponding probabilitys p_cats
    output is probability of specified event 'x' '''
    ind = np.where((cats==x).all(axis=1))[0][0]
    return p_cats[ind]


def data_gen(N, beta, seed):
    np.random.seed(seed)
    X0 = np.array([1.0]).repeat(N).reshape(-1, 1)
    X1 = np.random.binomial(1, 0.5, N).reshape(-1, 1)
    X2 = np.random.binomial(1, 0.5, N).reshape(-1, 1)
    X3 = np.random.binomial(1, 0.5, N).reshape(-1, 1)
    X4 = np.random.binomial(1, 0.5, N).reshape(-1, 1)
    X = np.concatenate([X0, X1, X2, X3, X4, X2*X3], 1)
    u =  0.5*np.random.randn(N)
    y = np.dot(X, beta) + u
    
    return X[:,1:-1], X, y, u  # first output is the variables without the constant and interaction


First intentionally misspecify the model by excluding the interaction term

In [3]:
num_loops = 500
k = 6
N = 5000
seed = 0
# set our query value of X=x:
cond_X_GT = np.array([1., 1., 0., 1., 0., 0.])  # note the last var has to be X2*X3

cond_X_est = cond_X_GT[1:-1]  # excludes offset and interaction. 
beta = np.array([3.3, 0.6, 0.5, 0.9, 0.6, 1.0])  # set beta/coefficients to some values
# we run multiple loops over a k-fold procedure to get an understanding of the variance
# in bias reduction.

biased_psis = []
upd_psis = []
for i in range(num_loops):
    if i % 50 == 0:
        print('Running loop:', i+1)
    seed += 1
    X, X_all, y, u = data_gen(N=N, beta=beta, seed=seed)  # generate the data
    # simulated data gen process (without noise) for purposes of finding E(Y|X=x)
    Psi_GT = np.dot(cond_X_GT, beta)
    
    kf = KFold(n_splits=k, shuffle=True)
    kf.get_n_splits(X)
    psis = []
    phis = []
    for train_index, test_index in kf.split(X):
        X_train = X[train_index]
        X_test = X[test_index]
        y_train = y[train_index]
        y_test = y[test_index]

        reg = LinearRegression().fit(X_train, y_train)
        psi = reg.predict((cond_X_est).reshape(1, -1))
        psis.append(psi)

        # return the set of possible outcomes, and their counts for X_ds1
        cats_train, counts_train = np.unique(X_train, axis=0,return_counts=True)
        p_cats_train = counts_train / len(X_train)  # this returns the normalized probabilities for each possible outcome

        all_phi = []
        for i in range(len(X_test)):
            y_tilde = y_test[i]
            x_tilde = X_test[i]
            EY_tildeX = reg.predict(x_tilde.reshape(1, -1))
            y_Ey = y_tilde - EY_tildeX
            if (x_tilde == cond_X_est).all():
                delta_over_fx = 1 / pmf(x_tilde, cats_train, p_cats_train)
            else:
                delta_over_fx = 0
            all_phi.append(delta_over_fx * y_Ey)

        phis = np.asarray(all_phi)
    phi = phis.mean()
    psi = np.asarray(psis).mean()    
    psi_est_updated = psi + phi
    biased_psis.append(psi)
    upd_psis.append(psi_est_updated)
biased_psis = np.asarray(biased_psis)
upd_psis = np.asarray(upd_psis)

Running loop: 1
Running loop: 51
Running loop: 101
Running loop: 151
Running loop: 201
Running loop: 251
Running loop: 301
Running loop: 351
Running loop: 401
Running loop: 451


In [4]:
print('Results for misspecified outcome model....')

print('True psi: ', Psi_GT)
print('naive psi: ', biased_psis.mean(), ' relative bias:',
      (biased_psis.mean() - Psi_GT)/Psi_GT * 100, '%')
print('updated TMLE psi: ', upd_psis.mean(), ' relative bias:',
      (upd_psis.mean() - Psi_GT)/Psi_GT * 100, '%')
print('Reduction in bias:', np.abs(biased_psis.mean() - Psi_GT)/Psi_GT * 100 - 
     np.abs(upd_psis.mean() - Psi_GT)/Psi_GT * 100, '%')

Results for misspecified outcome model....
True psi:  4.8
naive psi:  5.050779973366795  relative bias: 5.224582778474892 %
updated TMLE psi:  4.798730222584823  relative bias: -0.026453696149508765 %
Reduction in bias: 5.198129082325384 %


In [5]:
# This takes the reduction in relative bias for each simulation first, then takes an average
# (Owing to the nonlinearity of the ||x|| function, this gives different results which are
# worth considering.)
print('naive psi var:', biased_psis.var())
print('updated psi var:', upd_psis.var())
errors_naive = (biased_psis - Psi_GT)/Psi_GT *100
errors_updated = (upd_psis - Psi_GT)/Psi_GT *100
diff_errors = np.abs(errors_naive) - np.abs(errors_updated)
print('Average of reductions:', diff_errors.mean(), '%')

naive psi var: 0.00030687451683603507
updated psi var: 0.007279790011345534
Average of reductions: 3.836451852456813 %


Now let's try again, but correctly specifying the outcome model by including the interaction terms:

In [6]:

biased_psis = []
upd_psis = []
for i in range(num_loops):
    if i % 50 == 0:
        print('Running loop:', i+1)
    seed += 1
    X, X_all, y, u = data_gen(N=N, beta=beta, seed=seed) 
    # simulated data gen process (without noise) for purposes of finding E(Y|X=x)
    Psi_GT = np.dot(cond_X_GT, beta)
    
    kf = KFold(n_splits=k, shuffle=True)
    kf.get_n_splits(X_all)
    psis = []
    phis = []
    for train_index, test_index in kf.split(X):
        X_train = X_all[train_index, 1:]
        X_test = X_all[test_index, 1:]
        y_train = y[train_index]
        y_test = y[test_index]

        reg = LinearRegression().fit(X_train, y_train)
        psi = reg.predict((cond_X_GT[1:]).reshape(1, -1))
        psis.append(psi)

        # return the set of possible outcomes, and their counts for X_ds1
        cats_train, counts_train = np.unique(X_train, axis=0,return_counts=True)
        p_cats_train = counts_train / len(X_train)  # this returns the normalized probabilities for each possible outcome

        all_phi = []
        for i in range(len(X_test)):
            y_tilde = y_test[i]
            x_tilde = X_test[i]
            EY_tildeX = reg.predict(x_tilde.reshape(1, -1))
            y_Ey = y_tilde - EY_tildeX
            if (x_tilde == cond_X_GT[1:]).all():
                delta_over_fx = 1 / pmf(x_tilde, cats_train, p_cats_train)
            else:
                delta_over_fx = 0
            all_phi.append(delta_over_fx * y_Ey)

        phis = np.asarray(all_phi)
    phi = phis.mean()
    psi = np.asarray(psis).mean()    
    psi_est_updated = psi + phi
    biased_psis.append(psi)
    upd_psis.append(psi_est_updated)
biased_psis = np.asarray(biased_psis)
upd_psis = np.asarray(upd_psis)

Running loop: 1
Running loop: 51
Running loop: 101
Running loop: 151
Running loop: 201
Running loop: 251
Running loop: 301
Running loop: 351
Running loop: 401
Running loop: 451


In [7]:
print('Results for correctly specified outcome model....')

print('True psi: ', Psi_GT)
print('naive psi: ', biased_psis.mean(), ' relative bias:',
      (biased_psis.mean() - Psi_GT)/Psi_GT * 100, '%')
print('updated TMLE psi: ', upd_psis.mean(), ' relative bias:',
      (upd_psis.mean() - Psi_GT)/Psi_GT * 100, '%')
print('Reduction in bias:', np.abs(biased_psis.mean() - Psi_GT)/Psi_GT * 100 - 
     np.abs(upd_psis.mean() - Psi_GT)/Psi_GT * 100, '%')

Results for correctly specified outcome model....
True psi:  4.8
naive psi:  4.80026775343152  relative bias: 0.005578196489997023 %
updated TMLE psi:  4.7990347537751585  relative bias: -0.020109296350860156 %
Reduction in bias: -0.014531099860863133 %


In [8]:
# This takes the reduction in relative bias for each simulation first, then takes an average
# (Owing to the nonlinearity of the ||x|| function, this gives different results which are
# worth considering.)
print('naive psi var:', biased_psis.var())
print('updated psi var:', upd_psis.var())
errors_naive = (biased_psis - Psi_GT)/Psi_GT *100
errors_updated = (upd_psis - Psi_GT)/Psi_GT *100
diff_errors = np.abs(errors_naive) - np.abs(errors_updated)
print('Average of reductions:', diff_errors.mean(), '%')

naive psi var: 0.00031138339366269997
updated psi var: 0.005507836869718758
Average of reductions: -0.9150817980484894 %


Note that when the models have been correctly specified, the IF actually makes things worse, and the variance of the updated estimator is also higher than the naive estimator.